<a href="https://colab.research.google.com/github/henryfuentesbenito/andi2_challenge-/blob/main/Andy2_challenge_GraphFromTrajectoryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00


In [16]:
import glob
import os
import pandas as pd

def load_trajectory_data_from_folders(main_folder):
    all_trajectory_data = []
    folder_names = os.listdir(main_folder)
    for folder_name in folder_names:
        folder_path = os.path.join(main_folder, folder_name)
        if os.path.isdir(folder_path):
            file_paths = glob.glob(os.path.join(folder_path, '*.csv'))
            for file_path in file_paths:
                trajectory_data = pd.read_csv(file_path)
                all_trajectory_data.append(trajectory_data)
    return all_trajectory_data

# Ruta de la carpeta principal que contiene las subcarpetas con archivos csv
main_folder = '/content/drive/MyDrive/andi2_dataset6/ref/track_2'

# Cargar datos de trayectorias desde las subcarpetas con archivos csv
trajectory_data_list = load_trajectory_data_from_folders(main_folder)

In [4]:
len(trajectory_data_list)

60

In [15]:
trajectory_data_list[0]

,traj_idx,frame,x,y
0,0.0,117.0,207.293228,24.012350
1,0.0,118.0,207.455854,23.816246
2,0.0,119.0,207.221884,23.649520
3,0.0,120.0,208.964227,23.341389
4,0.0,121.0,208.716672,25.648786
...,...,...,...,...
5640,37.0,195.0,160.510163,94.636454
5641,37.0,196.0,159.001391,95.050213
5642,37.0,197.0,157.629891,95.083858
5643,37.0,198.0,157.493789,96.188235


______________________________________________________________________________________________________________
____________________________________________________________________________________________________
___________________________________________________________________________________________________________
________________________________________________________________________________________________________
____________________________________________________________________________________________________

**primer intento de codigo de la class GraphFromTrajectoryData modificado**

In [17]:
import numpy as np
import torch
from skimage import measure
from torch_geometric.data import Data

class GraphFromTrajectoryData:
    """Graph representation of particle trajectories."""

    def __init__(self, connectivity_radius, max_frame_distance):
        """Initialize graph constructor."""
        self.connectivity_radius = connectivity_radius
        self.max_frame_distance = max_frame_distance

    def __call__(self, trajectory):
        """Construct graph from a single trajectory."""
        graph = self.construct_graph(trajectory)
        return graph

    def construct_graph(self, trajectory):
        """Construct graph from a single trajectory."""
        x, edge_index, edge_attr = [], [], []

        # Compute node features
        features, index_labels = self.compute_node_features(trajectory)
        x.extend(features)
        node_indices = np.arange(len(features))

        # Compute connectivity
        edge_index, edge_attr = self.compute_connectivity(features, node_indices)

        x = torch.tensor(np.array(x), dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float).view(-1, 1)

        # Construct Data object
        graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

        return graph

    def compute_node_features(self, trajectory):
        """Compute node features."""
        features = trajectory[['x', 'y']].values  # Coordenadas de la trayectoria
        return features, [trajectory['traj_idx'].iloc[0]]

    def compute_connectivity(self, features, node_indices):
        """Compute connectivity."""
        edge_index = []
        edge_attr = []
        num_nodes = len(features)

        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                distance = np.linalg.norm(features[i] - features[j])
                if distance < self.connectivity_radius:
                    edge_index.append([node_indices[i], node_indices[j]])
                    edge_attr.append(distance)

        return edge_index, edge_attr


In [18]:
trajectory_graphs = []

# Aquí va el código para cargar los datos de trayectorias en `trajectory_data_list`

# Luego, se itera sobre cada DataFrame en la lista y construyes los grafos
for trajectory_data in trajectory_data_list:
    # Construir el grafo a partir de los datos de trayectoria
    graph = GraphFromTrajectoryData(connectivity_radius=0.6, max_frame_distance=2)(trajectory_data)
    trajectory_graphs.append(graph)

In [19]:
trajectory_graphs

[Data(x=[5519, 2], edge_index=[2, 4289], edge_attr=[4289, 1]),
 Data(x=[4919, 2], edge_index=[2, 5036], edge_attr=[5036, 1]),
 Data(x=[5739, 2], edge_index=[2, 4744], edge_attr=[4744, 1])]

__________________________________________________________________________________________
____________________________________________________________________________________________
_________________________________________________________________________________________
____________________________________________________________________________________________

**Codigo corregido con resultados adecuados**

In [20]:
import numpy as np
import torch
from skimage import measure
from torch_geometric.data import Data

class GraphFromTrajectoryData:
    """Graph representation of particle trajectories."""

    def __init__(self, connectivity_radius, max_frame_distance):
        """Initialize graph constructor."""
        self.connectivity_radius = connectivity_radius
        self.max_frame_distance = max_frame_distance

    def __call__(self, trajectory):
        """Construct graph from a single trajectory."""
        graph = self.construct_graph(trajectory)
        return graph

    def construct_graph(self, trajectory):
        """Construct graph from a single trajectory."""
        x, edge_index, edge_attr = [], [], []

        # Compute node features
        features, index_labels = self.compute_node_features(trajectory)
        x.extend(features)
        node_indices = np.arange(len(features))

        # Compute connectivity
        edge_index, edge_attr = self.compute_connectivity(features, node_indices)

        x = torch.tensor(np.array(x), dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float).view(-1, 1)

        # Construct Data object
        graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

        return graph



    def compute_node_features(self, trajectory):
      """Compute node features."""
      features = trajectory[['x', 'y', 'frame']].values  # Coordenadas de la trayectoria y número de fotograma
      return features, [trajectory['traj_idx'].iloc[0]]



    def compute_connectivity(self, features, node_indices):
        """Compute connectivity."""
        edge_index = []
        edge_attr = []
        num_nodes = len(features)

        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                distance = np.linalg.norm(features[i][:2] - features[j][:2])
                frame_distance = abs(features[i][2] - features[j][2])  # Diferencia entre fotogramas
                if distance < self.connectivity_radius and frame_distance <= self.max_frame_distance:
                    edge_index.append([node_indices[i], node_indices[j]])
                    edge_attr.append(distance)

        return edge_index, edge_attr


In [21]:
trajectory_graphs = []

# Iterar sobre cada DataFrame en la lista de datos de trayectorias
for trajectory_data in trajectory_data_list:
    # Construir el grafo a partir de los datos de trayectoria
    graph = GraphFromTrajectoryData(connectivity_radius=0.6, max_frame_distance=2)(trajectory_data)
    trajectory_graphs.append(graph)

In [23]:
trajectory_graphs[0]

Data(x=[5519, 3], edge_index=[2, 791], edge_attr=[791, 1])

In [7]:
trajectory_graphs_grouped = [[] for _ in range(len(trajectory_data_list))]

for i, trajectory_data in enumerate(trajectory_data_list):
    # Agrupar los datos de trayectorias por 'traj_idx'
    grouped_trajectories = trajectory_data.groupby('traj_idx')

    # Iterar sobre cada grupo de trayectorias
    for traj_idx, group in grouped_trajectories:
        # Construir el grafo para el grupo actual
        graph = GraphFromTrajectoryData(connectivity_radius=0.6, max_frame_distance=2)(group)
        trajectory_graphs_grouped[i].append((traj_idx, graph))

In [ ]:
trajectory_graphs_grouped

In [8]:
trajectory_graphs_grouped[0]

[(0.0, Data(x=[83, 3], edge_index=[2, 21], edge_attr=[21, 1])),
 (1.0, Data(x=[200, 3], edge_index=[2, 33], edge_attr=[33, 1])),
 (2.0, Data(x=[200, 3], edge_index=[2, 29], edge_attr=[29, 1])),
 (3.0, Data(x=[200, 3], edge_index=[2, 30], edge_attr=[30, 1])),
 (4.0, Data(x=[141, 3], edge_index=[2, 25], edge_attr=[25, 1])),
 (5.0, Data(x=[200, 3], edge_index=[2, 22], edge_attr=[22, 1])),
 (6.0, Data(x=[72, 3], edge_index=[2, 12], edge_attr=[12, 1])),
 (7.0, Data(x=[200, 3], edge_index=[2, 17], edge_attr=[17, 1])),
 (8.0, Data(x=[200, 3], edge_index=[2, 35], edge_attr=[35, 1])),
 (9.0, Data(x=[200, 3], edge_index=[2, 22], edge_attr=[22, 1])),
 (10.0, Data(x=[33, 3], edge_index=[2, 3], edge_attr=[3, 1])),
 (11.0, Data(x=[200, 3], edge_index=[2, 31], edge_attr=[31, 1])),
 (12.0, Data(x=[200, 3], edge_index=[2, 15], edge_attr=[15, 1])),
 (13.0, Data(x=[116, 3], edge_index=[2, 16], edge_attr=[16, 1])),
 (14.0, Data(x=[76, 3], edge_index=[2, 13], edge_attr=[13, 1])),
 (15.0, Data(x=[73, 3], ed

In [9]:
trajectory_graphs_grouped[1]

[(0.0, Data(x=[200, 3], edge_index=[2, 26], edge_attr=[26, 1])),
 (1.0, Data(x=[200, 3], edge_index=[2, 30], edge_attr=[30, 1])),
 (2.0, Data(x=[200, 3], edge_index=[2, 34], edge_attr=[34, 1])),
 (3.0, Data(x=[200, 3], edge_index=[2, 26], edge_attr=[26, 1])),
 (4.0, Data(x=[200, 3], edge_index=[2, 20], edge_attr=[20, 1])),
 (5.0, Data(x=[200, 3], edge_index=[2, 40], edge_attr=[40, 1])),
 (6.0, Data(x=[200, 3], edge_index=[2, 20], edge_attr=[20, 1])),
 (7.0, Data(x=[21, 3], edge_index=[2, 2], edge_attr=[2, 1])),
 (8.0, Data(x=[105, 3], edge_index=[2, 9], edge_attr=[9, 1])),
 (9.0, Data(x=[200, 3], edge_index=[2, 33], edge_attr=[33, 1])),
 (10.0, Data(x=[200, 3], edge_index=[2, 33], edge_attr=[33, 1])),
 (11.0, Data(x=[39, 3], edge_index=[2, 7], edge_attr=[7, 1])),
 (12.0, Data(x=[200, 3], edge_index=[2, 36], edge_attr=[36, 1])),
 (13.0, Data(x=[29, 3], edge_index=[2, 2], edge_attr=[2, 1])),
 (14.0, Data(x=[55, 3], edge_index=[2, 3], edge_attr=[3, 1])),
 (15.0, Data(x=[27, 3], edge_index

________________________________________________________________________________________________________________________
______________________________________________________________________________________________________________
________________________________________________________________________________________________________________
_______________________________________________________________________________________________________________
___________________________________________________________________________________________________
________________________________________________________________________________________________________________
_______________________________________________________________________________________________

** codigos de prueba**

In [49]:
trajectory_graphs = []

for trajectory_data in trajectory_data_list:
    # Agrupar los datos de trayectorias por 'traj_idx'
    grouped_trajectories = trajectory_data.groupby('traj_idx')

    # Iterar sobre cada grupo de trayectorias
    for traj_idx, group in grouped_trajectories:
        # Construir el grafo para el grupo actual
        graph = GraphFromTrajectoryData(connectivity_radius=0.6, max_frame_distance=2)(group)
        trajectory_graphs.append((traj_idx, graph))

In [55]:
trajectory_graphs[0]

[(0.0, Data(x=[200, 3], edge_index=[2, 32], edge_attr=[32, 1])),
 (1.0, Data(x=[200, 3], edge_index=[2, 29], edge_attr=[29, 1])),
 (2.0, Data(x=[45, 3], edge_index=[2, 6], edge_attr=[6, 1])),
 (3.0, Data(x=[148, 3], edge_index=[2, 21], edge_attr=[21, 1])),
 (4.0, Data(x=[200, 3], edge_index=[2, 19], edge_attr=[19, 1])),
 (5.0, Data(x=[63, 3], edge_index=[2, 8], edge_attr=[8, 1])),
 (6.0, Data(x=[188, 3], edge_index=[2, 35], edge_attr=[35, 1])),
 (7.0, Data(x=[200, 3], edge_index=[2, 27], edge_attr=[27, 1])),
 (8.0, Data(x=[200, 3], edge_index=[2, 28], edge_attr=[28, 1])),
 (9.0, Data(x=[102, 3], edge_index=[2, 11], edge_attr=[11, 1])),
 (10.0, Data(x=[200, 3], edge_index=[2, 19], edge_attr=[19, 1])),
 (11.0, Data(x=[131, 3], edge_index=[2, 21], edge_attr=[21, 1])),
 (12.0, Data(x=[200, 3], edge_index=[2, 25], edge_attr=[25, 1])),
 (13.0, Data(x=[50, 3], edge_index=[2, 5], edge_attr=[5, 1])),
 (14.0, Data(x=[200, 3], edge_index=[2, 17], edge_attr=[17, 1])),
 (15.0, Data(x=[200, 3], edge

In [11]:
trajectory_graphs = []

# Iterar sobre cada DataFrame en la lista de datos de trayectorias
for trajectory_data in trajectory_data_list:
    # Construir el grafo a partir de los datos de trayectoria
    graph = GraphFromTrajectoryData(connectivity_radius=0.6, max_frame_distance=2)(trajectory_data)
    trajectory_graphs.append(graph)

KeyboardInterrupt: 

In [44]:
len(trajectory_graphs)

3

In [45]:
trajectory_graphs

[Data(x=[5519, 3], edge_index=[2, 791], edge_attr=[791, 1]),
 Data(x=[4919, 3], edge_index=[2, 712], edge_attr=[712, 1]),
 Data(x=[5739, 3], edge_index=[2, 804], edge_attr=[804, 1])]